## Load libraries

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = 'last_expr'

In [ ]:
import os,sys
import re

sys.dont_write_bytecode = True

In [ ]:
import pandas as pd
# import geopandas as gpd
import joblib

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from skimage.color import rgb2gray

from pprint import pprint
from pathlib import Path

import pdb

### Import Holoviews

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

### Import PyTorch


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms
from torch.autograd import Variable

## Helpers

In [ ]:
def info(arr, header=None):
    if header is None:
        header = "="*30
    print(header)
    print("shape: ", arr.shape)
    print("dtype: ", arr.dtype)
    print("min, max: ", min(np.ravel(arr)), max(np.ravel(arr)))

## Set data path

### Sample datasets

In [ ]:
DATA_ROOT = Path('/data/hayley-old/faces-inria/')

In [ ]:
info(X)

In [ ]:
info(Y)

In [ ]:
poses = Y[:,-1]
print(poses[0])

In [ ]:
def get_img(i):
    return hv.Image(X[i]).options(cmap='gray')
dmap = hv.DynamicMap(get_img, kdims=['ind'])
dmap.redim.range(ind=(0, len(X)-1))

In [ ]:
hv.Image(X[-1]).options(cmap='gray')

In [ ]:
csv = pd.read_csv(DATA_ROOT/'dataGuide.csv')
csv = csv.drop(columns=csv.columns[0])
csv.head()

In [ ]:
len(csv)

In [ ]:
img_names = csv.iloc[:,0]
n_imgs = len(np.unique(img_names))
n_imgs

### Collect images to X. Collect postions into Y
Each image is rescaled to [0.,1.] 
- dtype: float64
- range: [0.,1.]
- shape: (250,250,1)


1. Encode position labels to integers

In [ ]:
poses = csv.pose.astype('category')
cats = poses.cat.categories
cat2ind = {cat:i for i,cat in enumerate(cats)}
cat2ind

In [ ]:
csv["pose_int"] = poses.map(cat2ind)

In [ ]:
for irow, row in csv.iterrows():
    if row.poses_int != 0:
        print(irow)
        print(row.ImageName, row.pose, row.pose_int)
        break

2. Collect images and their respective pose information

In [ ]:
IMG_H, IMG_W, IMG_C = 250,250,1
X = np.zeros((n_imgs, IMG_H, IMG_W, IMG_C), dtype=float) #in range[0,1]
Y = np.zeros(n_imgs, dtype=int) # each entry is a categorical variable for position type
for irow, row in csv.iterrows():
    name = row.ImageName
    pose_int = row.pose_int
    fpath = DATA_ROOT/f"Images/Images/{name}"

    if fpath.exists():
        im_rgb = plt.imread(fpath)
        im_gray = rgb2gray(im_rgb)[..., np.newaxis]
        if im_gray.shape != (IMG_H, IMG_W, IMG_C):
            print(fpath)
            raise ValueError(f'img size must be {[IMG_H, IMG_W, IMG_C]}')
        X[irow] = im_gray
        Y[irow] = pose_int
print(len(X))

Grab 
- personne01ddd{+,-}{60,15}+0.jpg
- personne01ddd+0{+,-}{60,15}.jpg

In [ ]:
a = np.random.randn(9, 6) + 1j*np.random.randn(9, 6)
b = np.random.randn(2, 7, 8, 3) + 1j*np.random.randn(2, 7, 8, 3)

In [ ]:
u, s, vh = np.linalg.svd(a, full_matrices=True)
u.shape, s.shape, vh.shape

np.allclose(a, np.dot(u[:, :6] * s, vh))

smat = np.zeros((9, 6), dtype=complex)
smat[:6, :6] = np.diag(s)
np.allclose(a, np.dot(u, np.dot(smat, vh)))
